In [8]:
import gdown

In [10]:
%cd ..
%pwd

/home/grkmkola/Desktop/Projects/mlops-proje/kidney-disease-classification


/home/grkmkola/miniconda3/envs/kidney/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/grkmkola/Desktop/Projects/mlops-proje/kidney-disease-classification'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [26]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
    ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([
            self.config.artifacts_root,
            self.config.data_ingestion.root_dir,
            self.config.data_ingestion.unzip_dir,
        ])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config 

In [28]:
ConfigurationManager().config

[2024-07-22 10:04:08,065: INFO: utils: yaml file config/config.yaml loaded successfully:]
[2024-07-22 10:04:08,067: INFO: utils: yaml file params.yaml loaded successfully:]
[2024-07-22 10:04:08,068: INFO: utils: created directory at: artifacts:]
[2024-07-22 10:04:08,069: INFO: utils: created directory at: artifacts/data_ingestion:]
[2024-07-22 10:04:08,069: INFO: utils: created directory at: artifacts/data_ingestion:]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1THszi4x1hnmqTwTrPuNDYpMS3SmkIhRe/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [34]:
import zipfile
import gdown
from cnnClassifier.utils import logger, get_size
import os

In [39]:
class DataIngestion:
    def __init__(
            self,
            config: DataIngestionConfig
        ) -> None:
        self.config = config

    def download_file(self):
        '''
        Description
            fetch data from the url
        '''

        
        try:
            dataset_url = self.config.source_url
            local_data_file = self.config.local_data_file

            if os.path.exists(local_data_file):
                logger.info(f'{local_data_file} is already exists')
                return
            logger.info(f'Downloading data from {dataset_url} into file {local_data_file}')
            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, local_data_file)
            logger.info(f'Downloaded data from {dataset_url} into file {local_data_file}')
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        '''
        Description
            extracts the zip file
        '''

        unzip_path = self.config.unzip_dir

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
        
        logger.info(f'{self.config.local_data_file} is successfully extracted to {unzip_path}')

In [40]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-22 10:12:15,227: INFO: utils: yaml file config/config.yaml loaded successfully:]
[2024-07-22 10:12:15,228: INFO: utils: yaml file params.yaml loaded successfully:]
[2024-07-22 10:12:15,229: INFO: utils: created directory at: artifacts:]
[2024-07-22 10:12:15,230: INFO: utils: created directory at: artifacts/data_ingestion:]
[2024-07-22 10:12:15,231: INFO: utils: created directory at: artifacts/data_ingestion:]
[2024-07-22 10:12:15,231: INFO: 2769369459: artifacts/data_ingestion/data.zip is already exists:]
[2024-07-22 10:12:26,391: INFO: 2769369459: artifacts/data_ingestion/data.zip is successfully extracted to artifacts/data_ingestion:]
